In [67]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [68]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, sys
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, root_mean_squared_error
project_root = os.path.abspath("..") 
if project_root not in sys.path:
    sys.path.append(project_root)
from src.preprocessing import clean_data
from src.baseline_model import train_baseline_model, evaluate_model

In [69]:
def create_feature_list(df_clean):
    print("Creating list of feature correlation to totalRent...")
    results = []

    for col in df_clean:
        if col in ['totalRent','baseRent','serviceCharge','baseRentRange']: continue

        df_tmp = df_clean.dropna(subset=[col, "totalRent"])
        X = df_tmp[[col]].to_numpy()
        y = df_tmp['totalRent'].to_numpy()
        model_full = LinearRegression().fit(X, y)
        y_predict = model_full.predict(X)
        r2_full = r2_score(y, y_predict)
        train_rmse = np.sqrt(root_mean_squared_error(y, y_predict))
        results.append((col, r2_full, train_rmse))

    results.sort(key=lambda x: x[1], reverse=True)
    print(f"{'Feature':<25} | {'R² Score':<10} | RMSE: ")
    print("-" * 40)
    for col, scorer2, scorermse in results:
        print(f"{col:<25} | {scorer2:.4f} | {scorermse:.4f}")
    return results


Iteratives Hinzufügen von Features in Regressionsmodell
Es wird jeweils das Feature genutzt welches im den größten neuen Mehrwert bringt

In [70]:
""" feature_cols_test = ['livingSpace','picturecount']
target_col = 'totalRent'

df_tmp_test = df_clean.dropna(subset=feature_cols_test + [target_col])
X_train_test = df_tmp_test[feature_cols_test].values
y_train_test = df_tmp_test[target_col].values

model_multi_test = LinearRegression()
model_multi_test.fit(X_train_test, y_train_test)

y_train_pred_test = model_multi_test.predict(X_train_test)
train_rmse_test = np.sqrt(root_mean_squared_error(y_train_test, y_train_pred_test))
train_r2_test = r2_score(y_train_test, y_train_pred_test)

print(f"R²: {train_r2_test}")
print(f"RMSE: {train_rmse_test}") """


' feature_cols_test = [\'livingSpace\',\'picturecount\']\ntarget_col = \'totalRent\'\n\ndf_tmp_test = df_clean.dropna(subset=feature_cols_test + [target_col])\nX_train_test = df_tmp_test[feature_cols_test].values\ny_train_test = df_tmp_test[target_col].values\n\nmodel_multi_test = LinearRegression()\nmodel_multi_test.fit(X_train_test, y_train_test)\n\ny_train_pred_test = model_multi_test.predict(X_train_test)\ntrain_rmse_test = np.sqrt(root_mean_squared_error(y_train_test, y_train_pred_test))\ntrain_r2_test = r2_score(y_train_test, y_train_pred_test)\n\nprint(f"R²: {train_r2_test}")\nprint(f"RMSE: {train_rmse_test}") '

In [ ]:
def feature_best_fit_list(df_clean, feature_cols, results):
    #print("Creating list for Features assendin by ascending R² value...")
    possible_features = []
    for col_name, scorer2, scorermse in results:
        if col_name in ['serviceCharge','baseRent'] + feature_cols:
            #print(f"{col_name} already in")
            continue
        feature_cols_tmp = feature_cols + [col_name]
        df_tmp = df_clean.dropna(subset=feature_cols_tmp + ['totalRent'])
        X_train = df_tmp[feature_cols_tmp].values
        y_train = df_tmp['totalRent'].values
        model_multi = LinearRegression()
        model_multi.fit(X_train, y_train)
        y_train_pred = model_multi.predict(X_train)
        train_rmse = np.sqrt(root_mean_squared_error(y_train, y_train_pred))
        train_r2 = r2_score(y_train, y_train_pred)
        possible_features.append((col_name, train_r2, train_rmse))

    possible_features.sort(key=lambda x: x[1], reverse=True)
    return possible_features   
    


Main Method

In [ ]:
df = pd.read_csv("../data/train.csv")
print("✅ Loaded dataset:", df.shape)
df_clean = clean_data(df)
feature_list = create_feature_list(df_clean)
print(f"Best allowed Startingfeature: {feature_list[0][0]}")
feature_red_list = [feature_list[0][0]]
stepwise_list = [feature_list[0]]

for i in range (15):
    new_feature_list = feature_best_fit_list(df_clean, feature_red_list, feature_list)
    #for col, a, b in new_feature_list:
        #print(f"{col}, {a}")
    stepwise_list.append(new_feature_list[0])
    feature_red_list.append(new_feature_list[0][0])
    print(f"Added feature {new_feature_list[0][0]}, all R^2: {new_feature_list[0][1]}, all RMSE: {new_feature_list[0][2]}")

""" for col in feature_red_list:
    print({col})   """  

for col, scorer2, scorermse in stepwise_list:
    print(f"{col:<25} | {scorer2:.4f} | {scorermse:.4f}")

✅ Loaded dataset: (346, 50)
Dataframe before cleaning:  (346, 50)
Cleaning Data....
Only regarding livingSpace > 10 m² & < 1000m² Shape:  (343, 50)
Removing PLZ that are not based in Münster
Missing Values serviceCharge 19
Missing values serviceCharge after filling: 0
Removing columns with to many NaNs
Dataframe nach dem Cleanen:  (342, 33)
Creating list of feature correlation to totalRent...
Feature                   | R² Score   | RMSE: 
----------------------------------------
livingSpace               | 0.7847 | 15.4056
americanArea              | 0.7847 | 15.4056
livingSpaceRange          | 0.7479 | 16.0257
noRooms                   | 0.5727 | 18.2854
noRoomsRange              | 0.5504 | 18.5197
regio3_num                | 0.3067 | 20.6376
interiorQual_num          | 0.1473 | 21.7336
picturecount              | 0.1468 | 21.7365
typeOfFlat_num            | 0.1230 | 21.8864
geo_plz                   | 0.1197 | 21.9070
condition_num             | 0.1066 | 21.9881
balcony_num         

Compare Complexity of the Model between Training and Validation Data set
Feature                   | R² Score   | RMSE: 
----------------------------------------
livingSpace               | 0.7847 | 15.4056
americanArea              | 0.7847 | 15.4056
livingSpaceRange          | 0.7479 | 16.0257
noRooms                   | 0.5727 | 18.2854
noRoomsRange              | 0.5504 | 18.5197
picturecount              | 0.1468 | 21.7365
geo_plz                   | 0.1197 | 21.9070
condition_num             | 0.1050 | 21.9983
balcony_num               | 0.0909 | 22.0840
heatingType_num           | 0.0567 | 22.2888
yearConstructedRange      | 0.0494 | 22.3322
thermalChar               | 0.0428 | 22.3707
newlyConst_num            | 0.0392 | 22.3915
cellar_num                | 0.0216 | 22.4937
firingTypes_num           | 0.0210 | 22.4968
lift_num                  | 0.0203 | 22.5012
garden_num                | 0.0190 | 22.5084
hasKitchen_num            | 0.0176 | 22.5164
lastRefurbish             | 0.0149 | 22.5319
numberOfFloors            | 0.0112 | 22.5532
scoutId                   | 0.0109 | 22.5551
yearConstructed           | 0.0076 | 22.5737
floor                     | 0.0065 | 22.5799
typeOfFlat_num            | 0.0060 | 22.5826
regio3_num                | 0.0040 | 22.5941
interiorQual_num          | 0.0037 | 22.5957
telekomUploadSpeed        | 0.0016 | 22.6075
noParkSpaces              | 0.0003 | 22.6147
pricetrend                | 0.0000 | 22.6167

In [ ]:
# ---------------------------------------------------------------------
# 2.1 Test: Cleaning results
# ---------------------------------------------------------------------
print("🔹 Running data cleaning test...")

try:
    df = pd.read_csv("../data/train.csv")
    df_clean = clean_data(df)

    n_rows = len(df_clean)
    n_numeric = len(df_clean.select_dtypes(include=[np.number]).columns)
    has_nan = df_clean.isnull().any().any()

    if n_rows > 300 and not has_nan and n_numeric > 10:
        print(f"✅ Cleaning successful: {n_rows} rows, {n_numeric} numeric features, no missing values.")
    else:
        print(f"⚠️ Cleaning incomplete — Rows: {n_rows}, Numeric cols: {n_numeric}, Missing values: {has_nan}")
except Exception as e:
    print(f"❌ Cleaning test failed: {e}")


# ---------------------------------------------------------------------
# 1.2 Test: Baseline model on 1D (livingSpace → totalRent)
# ---------------------------------------------------------------------
print("\n🔹 Running baseline model test...")

try:
    df_train = pd.read_csv("../data/train.csv")
    df_val = pd.read_csv("../data/validation.csv")

    df_train_clean = clean_data(df_train)
    df_val_clean = clean_data(df_val)

    X_train = df_train_clean[["livingSpace"]].to_numpy()
    y_train = df_train_clean["totalRent"].to_numpy()
    X_val = df_val_clean[["livingSpace"]].to_numpy()
    y_val = df_val_clean["totalRent"].to_numpy()

    model = train_baseline_model(X_train, y_train)
    eval_result = evaluate_model(model, X_val, y_val)

    r2 = float(eval_result["r2"])
    rmse = float(eval_result["rmse"])

    print(f"→ R²: {r2:.3f}, RMSE: {rmse:.1f} €")

    if 0.75 <= r2 and rmse <= 250:
        print("✅ Baseline model performance within expected range.")
    else:
        print("⚠️ Model performance outside expected range — check data cleaning or scaling.")
except Exception as e:
    print(f"❌ Baseline model test failed: {e}")


# ---------------------------------------------------------------------
# ✅ 1.3 Test: Contribution of categorical features
# ---------------------------------------------------------------------
print("\n🔹 Running categorical feature test...")

try:
    df = pd.read_csv("../data/train.csv")
    df_clean = clean_data(df)

    cat_cols = [c for c in df_clean.columns if c.endswith("_num")]

    if len(cat_cols) == 0:
        print("⚠️ No categorical _num features found after cleaning.")
    else:
        cat_feature = cat_cols[0]
        X = df_clean[[cat_feature]].to_numpy()
        y = df_clean["totalRent"].to_numpy()

        model = LinearRegression()
        model.fit(X, y)
        y_pred = model.predict(X)
        r2 = r2_score(y, y_pred)

        if r2 > 0.1:
            print(f"✅ Feature '{cat_feature}' contributes to prediction (R²={r2:.2f})")
        else:
            print(f"⚠️ Feature '{cat_feature}'.07) weak contribution (R²={r2:.2f})")
except Exception as e:
    print(f"❌ Categorical feature test failed: {e}")